In [1]:
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import grequests
import pandas as pd
import time
import random

C:\Users\efons\AppData\Local\Continuum\anaconda3\lib\site-packages\grequests.py:21: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['urllib3.util (C:\\Users\\efons\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\urllib3\\util\\__init__.py)', 'urllib3.contrib.pyopenssl (C:\\Users\\efons\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\urllib3\\contrib\\pyopenssl.py)']. 
  curious_george.patch_all(thread=False, select=False)
C:\Users\efons\AppData\Local\Continuum\anaconda3\lib\site-packages\gevent\hub.py:154: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
# number of pages are 0-3000 stepping by 120
pages = range(0,3000)
pages = list(pages[::120])
cl_post = []

## Scrape all page results for Bay Area

In [3]:
# web scrape all URLS to use later
start_time = time.time()
for page in pages:
    req = requests.get('https://sfbay.craigslist.org/search/apa?s=' + str(page))
    time.sleep(random.randint(1,3))
    
    # if error 
    if req.status_code != 200:
        print('Status_code not 200')
        break
    
    # parse the content
    html = BeautifulSoup(req.content, 'html5lib')
    
    # extract URLs loop over for each posting
    post = html.find_all('p', class_ = 'result-info')
    
    # Loop over post in each page. If post is blank stop if not append
    for i in range(0, 120):
        if post[i] != []:
            cl_post.append(post[i].a['href'])
        else:
            break
print("--- %s seconds ---" % (time.time() - start_time))

--- 90.78914022445679 seconds ---


In [4]:
# Making sure that we got 3000 URL
len(cl_post)

3000

In [5]:
# example of the first 10 urls scraped
cl_post[:10]

['https://sfbay.craigslist.org/pen/apa/d/menlo-park-prime-location-near-downtown/7086182317.html',
 'https://sfbay.craigslist.org/eby/apa/d/san-lorenzo-your-dream-home-is-about-to/7086182258.html',
 'https://sfbay.craigslist.org/eby/apa/d/oakland-stunning-views-of-the-bay/7079374661.html',
 'https://sfbay.craigslist.org/sby/apa/d/santa-clara-enjoy-2-weeks-free-lease/7086165212.html',
 'https://sfbay.craigslist.org/pen/apa/d/palo-alto-cozy-1-bed-1-bath-in/7086182002.html',
 'https://sfbay.craigslist.org/eby/apa/d/walnut-creek-courtyard-top-floor-two/7086181985.html',
 'https://sfbay.craigslist.org/sfc/apa/d/san-francisco-3br-2ba-spacious-urban/7086178232.html',
 'https://sfbay.craigslist.org/pen/apa/d/palo-alto-remodeled-home-on-large-lot/7086181902.html',
 'https://sfbay.craigslist.org/eby/apa/d/san-lorenzo-enjoy-the-luxury-of-this/7086181824.html',
 'https://sfbay.craigslist.org/eby/apa/d/alameda-1-bedroom-with-2-closets/7086181799.html']

Want to extract the area from url.

In [15]:
area = []
for post in cl_post:
    area.append(re.findall('/[a-z]{3}/', post)[0])

In [16]:
print(len(area))
area[:10]

3000


['/pen/',
 '/eby/',
 '/eby/',
 '/sby/',
 '/pen/',
 '/eby/',
 '/sfc/',
 '/pen/',
 '/eby/',
 '/eby/']

In [6]:
# Use URLs to scrape each postings html
htmls = []
start_time = time.time()
reqs = (grequests.get(link) for link in cl_post)
resp=grequests.imap(reqs, grequests.Pool(10))
 
for r in resp:
    html = BeautifulSoup(r.text, 'html.parser')
    htmls.append(html)
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 824.4546511173248 seconds ---


Even with asynchronous webscraping it still took about 14 mins get all the postings.

In [7]:
len(htmls)

3000

## Use HTML to get variables desired

In [17]:
# function to get titles
def get_titles(htmls):
    titles = []
    
    for html in htmls:
        title = html.h2.text.replace('\n', '')
        titles.append(title)
    return(titles)

In [18]:
def get_prices(htmls):
    prices = []
    
    # price
    for html in htmls:
        if html.find('span', class_ = 'price') is not None:
            price = html.find('span', class_ = 'price').text[1:]
            prices.append(price)
        else:
            price = np.nan
            prices.append(price)
    return(prices)

In [19]:
def get_rooms(htmls):
    bedrooms = []
    bathrooms = []
    
    for html in htmls:
        if 'BR' in str(html.find_all('span', class_ = 'shared-line-bubble')):
            [bedroom, bathroom] = html.find_all('span', class_ = 'shared-line-bubble')[0].text.split('/')
            bedroom = bedroom.replace(' ', '')[:-2]
            bathroom = bathroom.replace(' ', '')[:-2]
            
            bedrooms.append(bedroom)
            bathrooms.append(bathroom)
        else:
            bedroom = np.nan
            bathroom = np.nan
            
            bedrooms.append(bedroom)
            bathrooms.append(bathroom)
    return(bedrooms, bathrooms)

In [20]:
def get_sqft(htmls):
    areas = []
    
     # area. not alwasy specified
    for html in htmls:
        if html.find('span', class_ = 'housing') is not None:
        # if square footage even in title
            if 'ft' in str(html.find('span', class_ = 'housing')):
                sqr_ft = re.findall('[0-9]*ft', str(html.find('span', class_ = 'housing').text))[0][:-2]
                areas.append(sqr_ft)
            else:
                sqr_ft = np.nan
                areas.append(sqr_ft)
        else:
            sqr_ft = np.nan
            areas.append(sqr_ft)
    return(areas)

In [21]:
def time_posted(htmls):
    times = []
    
    for html in htmls:
        if html.time is not None:
            time = html.time.text.replace(' ', '')
            time = time.replace('\n', '')
            times.append(time)
        else:
            time = np.nan
            times.append(time)
    return(times)

In [22]:
def get_coord(htmls):
    lats = []
    longs = []
    
    # Location. not always mapped
    for html in htmls:
        if html.find('div', {'id': 'map'}) is not None:
            lat = html.find('div', {'id': 'map'})['data-latitude']
            long = html.find('div', {'id': 'map'})['data-longitude']
            lats.append(lat)
            longs.append(long)
        else:
            lat = np.nan
            long = np.nan

            lats.append(lat)
            longs.append(long)
    return(lats, longs)

In [23]:
def get_attrs(htmls):
    dogs = []
    cats = []
    parkings = []
    laundrys = []
    smokings = []
    
    for html in htmls:
        if html.find_all('p', class_ = 'attrgroup') is not None:
            cat = re.findall('[' ', \w]*cats[' ', \w]*', str(html.find_all('p', class_ = 'attrgroup')))
            dog = re.findall('[' ', \w]*dogs[' ', \w]*', str(html.find_all('p', class_ = 'attrgroup')))
            parking = re.findall('[' ', \w]*parking[' ', \w]*', str(html.find_all('p', class_ = 'attrgroup')))
            smoking = re.findall('[' ', \w]*smoking[' ', \w]*', str(html.find_all('p', class_ = 'attrgroup')))
        
        cats.append(cat)
        dogs.append(dog)
        parkings.append(parking)
        smokings.append(smoking)
    return(dogs,cats, parkings, smokings)

In [24]:
titles = get_titles(htmls)

In [25]:
prices = get_prices(htmls)

In [26]:
bedrooms, bathrooms = get_rooms(htmls)

In [27]:
sqft = get_sqft(htmls)

In [28]:
time = time_posted(htmls)

In [29]:
lats, longs = get_coord(htmls)

In [30]:
dogs, cats, parkings, smokings = get_attrs(htmls)

## Create DataFrame and Clean it

In [31]:
df = pd.DataFrame({
    'Title': titles,
    'Price': prices,
    'Bedrooms': bedrooms,
    'Bathrooms': bathrooms,
    'Square_feet': sqft,
    'Dog': dogs,
    'Cat': cats,
    'Parking': parkings,
    'Smoking': smokings,
    'Time_posted': time,
    'Longitude': longs,
    'Latitude': lats,
    'Area': area
})

In [32]:
df.head(10)

,Title,Price,Bedrooms,Bathrooms,Square_feet,Dog,Cat,Parking,Smoking,Time_posted,Longitude,Latitude,Area
0,$7500 / 4br - 2200ft2 - Prime Location Near Do...,7500,4,2,2200,[],[],[],[no smoking],2020-03-0218:03,-122.188886,37.451399,/pen/
1,$2195 / 1br - 950ft2 - YOUR DREAM HOME IS ABOU...,2195,1,1,950,[],[cats are OK ],[],[no smoking],2020-03-0218:03,-122.139221,37.686232,/eby/
2,$5375 / 3br - 1286ft2 - Stunning Views of the ...,5375,3,2,1286,[dogs are OK ],[cats are OK ],[],[],2020-02-2018:13,-122.274055,37.795558,/eby/
3,$3864 / 2br - 1149ft2 - Enjoy 2 Weeks FREE- Le...,3864,2,2,1149,[dogs are OK ],[cats are OK ],[],[no smoking],2020-03-0217:27,-121.984400,37.348300,/sby/
4,$2400 / 1br - 550ft2 - Cozy 1 bed/1 bath in Co...,2400,1,1,550,[],[],[],[no smoking],2020-03-0218:02,-122.152399,37.460140,/pen/
5,$2800 / 2br - 1061ft2 - Courtyard Top Floor Tw...,2800,2,2,1061,[dogs are OK ],[cats are OK ],[],[no smoking],2020-03-0218:02,-122.057397,37.901322,/eby/
6,"$4000 / 3br - 800ft2 - 3BR, 2BA: Spacious, Urb...",4000,3,2,800,[],[],[street parking],[no smoking],2020-03-0217:54,-122.407001,37.795077,/sfc/
7,$6850 / 2br - 1300ft2 - Remodeled Home on Larg...,6850,2,2.5,1300,[],[],[street parking],[no smoking],2020-03-0218:02,-122.145085,37.434029,/pen/
8,$2450 / 2br - 1000ft2 - ENJOY THE LUXURY OF TH...,2450,2,1,1000,[],[cats are OK ],[],[no smoking],2020-03-0218:02,-122.139221,37.686232,/eby/
9,$2705 / 1br - 770ft2 - 1-Bedroom with 2 Closet...,2705,1,1,770,[dogs are OK ],[cats are OK ],[],[no smoking],2020-03-0218:02,-122.272470,37.762614,/eby/


There are post that got deleted which means I have to remove those.

In [33]:
df = df[~df['Title'].str.contains('deleted')]
df = df.reset_index(drop=True)

In [34]:
# there were 3 post that got deleted
len(df)

2999

In [35]:
# Would like to remove all the brackets inside my dataframe
clean = ['Dog', 'Cat', 'Parking', 'Smoking']
for a in clean:
    df[a] = df[a].str[0]

In [36]:
df.head()

,Title,Price,Bedrooms,Bathrooms,Square_feet,Dog,Cat,Parking,Smoking,Time_posted,Longitude,Latitude,Area
0,$7500 / 4br - 2200ft2 - Prime Location Near Do...,7500,4,2,2200,NaN,NaN,NaN,no smoking,2020-03-0218:03,-122.188886,37.451399,/pen/
1,$2195 / 1br - 950ft2 - YOUR DREAM HOME IS ABOU...,2195,1,1,950,NaN,cats are OK,NaN,no smoking,2020-03-0218:03,-122.139221,37.686232,/eby/
2,$5375 / 3br - 1286ft2 - Stunning Views of the ...,5375,3,2,1286,dogs are OK,cats are OK,NaN,NaN,2020-02-2018:13,-122.274055,37.795558,/eby/
3,$3864 / 2br - 1149ft2 - Enjoy 2 Weeks FREE- Le...,3864,2,2,1149,dogs are OK,cats are OK,NaN,no smoking,2020-03-0217:27,-121.984400,37.348300,/sby/
4,$2400 / 1br - 550ft2 - Cozy 1 bed/1 bath in Co...,2400,1,1,550,NaN,NaN,NaN,no smoking,2020-03-0218:02,-122.152399,37.460140,/pen/


In [38]:
df['Area'] = df['Area'].str.strip('/')

In [39]:
df.head()

,Title,Price,Bedrooms,Bathrooms,Square_feet,Dog,Cat,Parking,Smoking,Time_posted,Longitude,Latitude,Area
0,$7500 / 4br - 2200ft2 - Prime Location Near Do...,7500,4,2,2200,NaN,NaN,NaN,no smoking,2020-03-0218:03,-122.188886,37.451399,pen
1,$2195 / 1br - 950ft2 - YOUR DREAM HOME IS ABOU...,2195,1,1,950,NaN,cats are OK,NaN,no smoking,2020-03-0218:03,-122.139221,37.686232,eby
2,$5375 / 3br - 1286ft2 - Stunning Views of the ...,5375,3,2,1286,dogs are OK,cats are OK,NaN,NaN,2020-02-2018:13,-122.274055,37.795558,eby
3,$3864 / 2br - 1149ft2 - Enjoy 2 Weeks FREE- Le...,3864,2,2,1149,dogs are OK,cats are OK,NaN,no smoking,2020-03-0217:27,-121.984400,37.348300,sby
4,$2400 / 1br - 550ft2 - Cozy 1 bed/1 bath in Co...,2400,1,1,550,NaN,NaN,NaN,no smoking,2020-03-0218:02,-122.152399,37.460140,pen


In [40]:
# lets see how many values were missing
df.isna().sum()

Title             0
Price             6
Bedrooms         96
Bathrooms        96
Square_feet     799
Dog            1361
Cat            1155
Parking        2379
Smoking        1060
Time_posted       1
Longitude        16
Latitude         16
Area              0
dtype: int64

There alot of variables missing, but we can fill some of them up by assuming that if the postings didn't specify that dogs were ok meant that they were not ok.

In [41]:
# Im going to guess that if they didn't clarify that dogs/cats are ok that means they are not ok
df['Dog'] = df['Dog'].str.contains('OK')
df['Dog'] = df['Dog'].replace(True, 'Yes')
df['Dog'] = df['Dog'].replace(np.nan, 'No')

In [42]:
df['Cat'] = df['Cat'].str.contains('OK')
df['Cat'] = df['Cat'].replace(True, 'Yes')
df['Cat'] = df['Cat'].replace(np.nan, 'No')

In [43]:
# Assume if poster didn't put no smoking then smoking is ok
df['Smoking'] = df['Smoking'].replace(np.nan, 'smoking ok')

In [44]:
# there are string values in number of bathrooms
df['Bathrooms'].value_counts()

1         1906
2          695
1.5        123
2.5         88
3           47
3.5         13
0            9
4            8
split        5
4.5          3
shared       3
5.5          2
5            1
Name: Bathrooms, dtype: int64

In [45]:
df['Bathrooms'] = df['Bathrooms'].replace(['shared', 'split'], .5)

In [46]:
# change datatypes to floats
df[['Price', 'Bedrooms', 'Bathrooms', 'Longitude', 'Latitude', 'Square_feet']] = df[['Price', 'Bedrooms', 'Bathrooms','Longitude', 'Latitude', 'Square_feet']].astype('float64')

In [47]:
# Make column time stamp
df['Time_posted'] = pd.to_datetime(df['Time_posted'],format='%Y-%m-%d%H:%M')

In [48]:
# Our results. readyt to analyze
df.head(10)

,Title,Price,Bedrooms,Bathrooms,Square_feet,Dog,Cat,Parking,Smoking,Time_posted,Longitude,Latitude,Area
0,$7500 / 4br - 2200ft2 - Prime Location Near Do...,7500.0,4.0,2.0,2200.0,No,No,NaN,no smoking,2020-03-02 18:03:00,-122.188886,37.451399,pen
1,$2195 / 1br - 950ft2 - YOUR DREAM HOME IS ABOU...,2195.0,1.0,1.0,950.0,No,Yes,NaN,no smoking,2020-03-02 18:03:00,-122.139221,37.686232,eby
2,$5375 / 3br - 1286ft2 - Stunning Views of the ...,5375.0,3.0,2.0,1286.0,Yes,Yes,NaN,smoking ok,2020-02-20 18:13:00,-122.274055,37.795558,eby
3,$3864 / 2br - 1149ft2 - Enjoy 2 Weeks FREE- Le...,3864.0,2.0,2.0,1149.0,Yes,Yes,NaN,no smoking,2020-03-02 17:27:00,-121.984400,37.348300,sby
4,$2400 / 1br - 550ft2 - Cozy 1 bed/1 bath in Co...,2400.0,1.0,1.0,550.0,No,No,NaN,no smoking,2020-03-02 18:02:00,-122.152399,37.460140,pen
5,$2800 / 2br - 1061ft2 - Courtyard Top Floor Tw...,2800.0,2.0,2.0,1061.0,Yes,Yes,NaN,no smoking,2020-03-02 18:02:00,-122.057397,37.901322,eby
6,"$4000 / 3br - 800ft2 - 3BR, 2BA: Spacious, Urb...",4000.0,3.0,2.0,800.0,No,No,street parking,no smoking,2020-03-02 17:54:00,-122.407001,37.795077,sfc
7,$6850 / 2br - 1300ft2 - Remodeled Home on Larg...,6850.0,2.0,2.5,1300.0,No,No,street parking,no smoking,2020-03-02 18:02:00,-122.145085,37.434029,pen
8,$2450 / 2br - 1000ft2 - ENJOY THE LUXURY OF TH...,2450.0,2.0,1.0,1000.0,No,Yes,NaN,no smoking,2020-03-02 18:02:00,-122.139221,37.686232,eby
9,$2705 / 1br - 770ft2 - 1-Bedroom with 2 Closet...,2705.0,1.0,1.0,770.0,Yes,Yes,NaN,no smoking,2020-03-02 18:02:00,-122.272470,37.762614,eby


In [49]:
# save csv to work EDA on anothe notebooks
df.to_csv('Craigslist.csv', index = False)